In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b7a7bdceeede3a385e46da0cb061f278186a874a68e010177031588219606d03
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
!pip install imbalanced-learn

In [3]:
from imblearn.over_sampling import RandomOverSampler

In [4]:
# Biblioteca que trata dados desbalanceados através da técnica de oversampling
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [5]:
import pandas as pd
import numpy as np

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *



In [8]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ML-banking").getOrCreate()

# dyf = glueContext.create_dynamic_frame.from_catalog(database='banking_tables', table_name='bank_costumer')
# dyf.printSchema()

# df = dyf.toDF()

In [12]:

df = spark.read.parquet("/content/run-1714852851989-part-block-0-r-00000-snappy.parquet", inferSchema = True)


In [13]:
df = df.toPandas()

In [14]:
# Selecionando as colunas relevantes
columns = ['creditscore', 'geography_France', 'geography_Germany', 'geography_Spain', 'gender_Female', 'gender_Male', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard', 'estimatedsalary', 'exited']
columns = [c for c in columns if c not in ['exited']]  # Removendo a coluna 'Exited' das features

# Separando a variável alvo (y) e as features (x)
y = df['exited']  # Variável alvo original
x = df[columns]   # Features originais

# Criando uma instância do RandomOverSampler para lidar com o desbalanceamento
ros = RandomOverSampler(sampling_strategy='minority')

# Aplicando oversampling às features e à variável alvo
x_train_balanced, y_train_balanced = ros.fit_resample(x, y)

# Exibindo informações sobre a distribuição da variável alvo antes e depois do balanceamento
print(f"Imbalanced target class:\n{Counter(y)}\n\nBalanced target class:\n{Counter(y_train_balanced)}\n")

# Calculando o número de novos pontos selecionados aleatoriamente durante o oversampling
print(x_train_balanced.shape[0] - df.shape[0], 'new random picked points')

Imbalanced target class:
Counter({0: 7960, 1: 2037})

Balanced target class:
Counter({0: 7960, 1: 7960})

5923 new random picked points


In [16]:
df

,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,geography_,geography_France,geography_Germany,geography_Spain,gender_Female,gender_Male
0,692,43.0,2,69014.49,2,0.0,0.0,164621.43,0,False,False,True,False,True,False
1,850,40.0,1,76914.21,1,1.0,0.0,174183.44,0,False,True,False,False,False,True
2,526,43.0,3,0.00,2,1.0,0.0,31705.19,0,False,False,False,True,True,False
3,806,31.0,10,138653.51,1,1.0,0.0,190803.37,0,False,False,True,False,False,True
4,745,44.0,0,119638.21,1,1.0,1.0,34265.08,1,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,569,43.0,8,161546.68,2,0.0,1.0,178187.28,0,False,False,False,True,False,True
9993,612,63.0,2,131629.17,2,1.0,0.0,122109.58,1,False,False,False,True,True,False
9994,627,62.0,3,143426.34,2,1.0,1.0,143104.30,0,False,False,True,False,False,True
9995,684,33.0,4,140700.61,1,1.0,0.0,103557.93,0,False,True,False,False,False,True


In [15]:
# Separando os dados de treino e de teste

x_train,x_test,y_train,y_test = train_test_split(x_train_balanced, y_train_balanced,test_size=0.3, random_state = 1) #70 % of train data, 30 % of test data
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((11144, 12), (4776, 12), (11144,), (4776,))

In [17]:
#Matriz de correlação

threshold = 0.9 #limite

# Matriz de correlação em valor absoluto
corr_matrix = x_train.corr().abs()
corr_matrix.head(len(columns)+2)

,creditscore,geography_France,geography_Germany,geography_Spain,gender_Female,gender_Male,age,tenure,balance,numofproducts,hascrcard,estimatedsalary
creditscore,1.000000,0.016007,0.009183,0.008495,0.004789,0.004789,0.017965,0.016953,0.012238,0.017047,0.000570,0.024480
geography_France,0.016007,1.000000,0.618884,0.504191,0.012397,0.012397,0.046495,0.007734,0.262522,0.008412,0.005889,0.000550
geography_Germany,0.009183,0.618884,1.000000,0.366070,0.030922,0.030922,0.073668,0.013623,0.420966,0.032584,0.021695,0.006792
geography_Spain,0.008495,0.504191,0.366070,1.000000,0.019101,0.019101,0.026089,0.024293,0.151518,0.025707,0.030969,0.008455
gender_Female,0.004789,0.012397,0.030922,0.019101,1.000000,1.000000,0.035320,0.004550,0.001808,0.036187,0.006750,0.020552
gender_Male,0.004789,0.012397,0.030922,0.019101,1.000000,1.000000,0.035320,0.004550,0.001808,0.036187,0.006750,0.020552
age,0.017965,0.046495,0.073668,0.026089,0.035320,0.035320,1.000000,0.016787,0.048358,0.026617,0.021072,0.000674
tenure,0.016953,0.007734,0.013623,0.024293,0.004550,0.004550,0.016787,1.000000,0.009012,0.012182,0.018229,0.027539
balance,0.012238,0.262522,0.420966,0.151518,0.001808,0.001808,0.048358,0.009012,1.000000,0.180633,0.017234,0.012741
numofproducts,0.017047,0.008412,0.032584,0.025707,0.036187,0.036187,0.026617,0.012182,0.180633,1.000000,0.001790,0.017758


In [ ]:
# Matriz triangular de correlações
"""
Matriz triangular superior de correlações a partir da matriz de correlação original.
Esse processo ajuda a identificar pares de variáveis altamente correlacionadas
"""
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
upper.head(len(columns)+2)

In [ ]:
# Seleciona colunas com correlações acima do limite
drop_cols = [column for column in upper.columns if any(upper[column] > threshold)]
print('Existem %d colunas para remover.' % (len(drop_cols)), 'No caso as colunas:', str(drop_cols))

In [20]:
# Removendo colunas identificadas como tendo correlações acima do limite especificado,
# tanto do conjunto de treinamento (x_train) quanto do conjunto de teste (x_test). Isso é feito para evitar multicolinearidade e melhorar a qualidade do modelo
x_train = x_train.drop(columns = drop_cols)
x_test = x_test.drop(columns = drop_cols)
print('Formato de X_train:', x_train.shape)
print('Formato de X_test:', x_test.shape)

Formato de X_train: (11144, 11)
Formato de X_test: (4776, 11)


## MODELOS DE APRENDIZADO DE MÁQUINA

KNN (K-Nearest Neighbors)

In [21]:
def get_scores(y, y_pred, col):
    # Calcula as métricas de desempenho
    data = {
        'Acurácia': np.round(accuracy_score(y, y_pred), 2),
        'Precisão': np.round(precision_score(y, y_pred), 2),
        'Recall': np.round(recall_score(y, y_pred), 2),
        'F1': np.round(f1_score(y, y_pred), 2),
        'ROC AUC': np.round(roc_auc_score(y, y_pred), 2)
    }

    # Cria um DataFrame com as métricas
    scores_df = pd.Series(data).to_frame(col)

    return scores_df

In [26]:

# Implementação do algoritmo K-NN
knnc = KNeighborsClassifier(n_neighbors=5)
knnc.fit(x_train, y_train)
y_pred_knnc = knnc.predict(x_test)

# Calcula a Matriz de Confusão
cf_matrix_knnc = confusion_matrix(y_test, y_pred_knnc)

# Calcula as probabilidades para a classe positiva
y_probs_knnc = knnc.predict_proba(x_test)[:, 1]

# Calcula as curvas ROC e AUC
fpr, tpr, thresholds = roc_curve(y_test, y_probs_knnc)
roc_auc = auc(fpr, tpr)

# Valores das métricas
accuracy = (y_test, y_pred_knnc)
precision = precision_score(y_test, y_pred_knnc)
recall = recall_score(y_test, y_pred_knnc)
f1 = f1_score(y_test, y_pred_knnc)

# Avaliação do K-NN: impressão das métricas
print(classification_report(y_test, y_pred_knnc))

# Imprime as métricas individualmente
print('Accuracy Score : ' + str(round(accuracy_score(y_test, y_pred_knnc), 3)))
print('Precision Score : ' + str(round(precision_score(y_test, y_pred_knnc), 3)))
print('Recall Score : ' + str(round(recall_score(y_test, y_pred_knnc), 3)))
print('F-Score : ' + str(round(f1_score(y_test, y_pred_knnc), 3)))

              precision    recall  f1-score   support

           0       0.71      0.58      0.64      2369
           1       0.65      0.77      0.70      2407

    accuracy                           0.67      4776
   macro avg       0.68      0.67      0.67      4776
weighted avg       0.68      0.67      0.67      4776

Accuracy Score : 0.673
Precision Score : 0.648
Recall Score : 0.768
F-Score : 0.703


In [27]:
from sklearn.naive_bayes import GaussianNB

# Implementação do algoritmo Naive Bayes (Gaussian)
nb_classifier = GaussianNB()
nb_classifier.fit(x_train, y_train)
y_pred_nb = nb_classifier.predict(x_test)

# Calcula a Matriz de Confusão
cf_matrix_nb = confusion_matrix(y_test, y_pred_nb)

# Valores das métricas
accuracy = accuracy_score(y_test, y_pred_nb)
precision = precision_score(y_test, y_pred_nb)
recall = recall_score(y_test, y_pred_nb)
f1 = f1_score(y_test, y_pred_nb)

# Avaliação do naive_bayes: impressão das métricas
print(classification_report(y_test, y_pred_nb))

# Imprime as métricas individualmente
print('Accuracy Score : ' + str(round(accuracy_score(y_test, y_pred_nb), 3)))
print('Precision Score : ' + str(round(precision_score(y_test, y_pred_nb), 3)))
print('Recall Score : ' + str(round(recall_score(y_test, y_pred_nb), 3)))
print('F-Score : ' + str(round(f1_score(y_test, y_pred_nb), 3)))


              precision    recall  f1-score   support

           0       0.68      0.73      0.70      2369
           1       0.71      0.66      0.68      2407

    accuracy                           0.69      4776
   macro avg       0.69      0.69      0.69      4776
weighted avg       0.69      0.69      0.69      4776

Accuracy Score : 0.692
Precision Score : 0.71
Recall Score : 0.657
F-Score : 0.682


In [28]:
from xgboost import XGBClassifier

# Implementação do algoritmo XGBClassifier
xgbc = XGBClassifier(use_label_encoder=False)
xgbc.fit(x_train, y_train)
y_pred_xgbc = xgbc.predict(x_test)

# Avaliação do XGBClassifier: impressão da matriz de confusão
cf_matrix_xgbc = confusion_matrix(y_test, y_pred_xgbc)
print(cf_matrix_xgbc)

# Calculando as probabilidades para a classe positiva
y_probs_xgbc = xgbc.predict_proba(x_test)[:, 1]

# Valores das métricas
accuracy = accuracy_score(y_test, y_pred_xgbc)
precision = precision_score(y_test, y_pred_xgbc)
recall = recall_score(y_test, y_pred_xgbc)
f1 = f1_score(y_test, y_pred_xgbc)

# Avaliação do xgboost: impressão das métricas
print(classification_report(y_test, y_pred_xgbc))

# Imprime as métricas individualmente
print('Accuracy Score : ' + str(round(accuracy_score(y_test, y_pred_xgbc), 3)))
print('Precision Score : ' + str(round(precision_score(y_test, y_pred_xgbc), 3)))
print('Recall Score : ' + str(round(recall_score(y_test, y_pred_xgbc), 3)))
print('F-Score : ' + str(round(f1_score(y_test, y_pred_xgbc), 3)))

[[2003  366]
 [ 212 2195]]
              precision    recall  f1-score   support

           0       0.90      0.85      0.87      2369
           1       0.86      0.91      0.88      2407

    accuracy                           0.88      4776
   macro avg       0.88      0.88      0.88      4776
weighted avg       0.88      0.88      0.88      4776

Accuracy Score : 0.879
Precision Score : 0.857
Recall Score : 0.912
F-Score : 0.884
